In [1]:
import json
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
df = pd.read_json('./data/v2/tsa_processed.json')
print('lines of data: ',len(df))
df.head()

lines of data:  444


,annotator_id,document_id,paragraph_index,text,annotation,annotation_raw
0,tsa,2005.00512,0,Extracting information about entities and thei...,"{'entities': {'a1': {'id': 'a1', 'type': 'a', ...",[{'@context': 'http://www.w3.org/ns/anno.jsonl...
1,tsa,2005.00512,1,Creating datasets for information extraction a...,"{'entities': {'a2': {'id': 'a2', 'type': 'a', ...",[{'@context': 'http://www.w3.org/ns/anno.jsonl...
2,tsa,2005.00512,3,To overcome the annotation challenges for larg...,"{'entities': {'a5': {'id': 'a5', 'type': 'a', ...",[{'@context': 'http://www.w3.org/ns/anno.jsonl...
3,tsa,2005.00512,6,"we introduce SciREX, a dataset that evaluates\...","{'entities': {'a4': {'id': 'a4', 'type': 'a', ...",[{'@context': 'http://www.w3.org/ns/anno.jsonl...
4,tsa,2005.00512,7,"In recent years, there has been multiple attem...","{'entities': {}, 'relations': {}}",[]


In [3]:
counts = df['document_id'].value_counts()
print('number and distribution of document:\n', counts)

number and distribution of document:
 2211.14208    69
2005.00512    61
2210.10073    51
2212.06522    51
1808.09602    48
2205.02033    44
2211.15202    37
2005.11184    35
2203.05325    30
2211.15196    18
Name: document_id, dtype: int64


In [4]:
# only save the dataframe with at least one relation
joint_dataset = {}

num_parg = 0
num_rels = 0

for idx in df.index:
    text = df['text'][idx] 
    annotation = df['annotation'][idx]
    relations = annotation['relations']

    if relations:
        # save
        id = str(df['document_id'][idx]) + '_paragraph' + str(df['paragraph_index'][idx])
        info = {
                'id': id,
                #'phase': 'train',
                #'paragraph': df['paragraph_index'][idx],
                'text': df['text'][idx],
                'entity': annotation['entities'],
                'relation': annotation['relations'],
        }
        joint_dataset[id] = info
        num_parg += 1
        num_rels += len(relations)

print('number of paragraph with relations: ',num_parg)
print('number of relations: ',num_rels)

# sampling from none 'relation': 

# None as a type? nearly half
# joint_dataset

number of paragraph with relations:  27
number of relations:  212


In [5]:
# len(joint_dataset)
# define the 3 type of relation: ap, pv, vc
# relation_entity_dic = {}

rel_to_entity = defaultdict(list)

entity_pair_dict = {}
st_relation_list = []

entity_type = defaultdict(list)

for paragraph in joint_dataset.keys():
    for rel in joint_dataset[paragraph]['relation']:
        text = joint_dataset[paragraph]['text']
        text = ' '.join(text.splitlines())
        
        rel_context = joint_dataset[paragraph]['relation'][rel]

        # entity name (like entity type v1 p1 a1...)
        source = rel_context['source']
        target = rel_context['target']
        # entity informstion 
        entity_context = joint_dataset[paragraph]['entity']
        s_entity = entity_context[source]
        t_entity = entity_context[target]
        st_relation = target[0] + source[0]

        # entity name
        s_positon_start = s_entity['surface_forms'][0]['start']
        s_positon_end = s_entity['surface_forms'][0]['end']
        t_positon_start = t_entity['surface_forms'][0]['start']
        t_positon_end = t_entity['surface_forms'][0]['end']
        s_name = text[s_positon_start: s_positon_end] 
        t_name = text[t_positon_start: t_positon_end]
        entity_pair = (t_name, s_name)
        entity_pair_dict[entity_pair] = st_relation
        rel_to_entity[st_relation].append(entity_pair)

        entity_type[source[0]].append(s_name) 
        entity_type[target[0]].append(t_name)

        print(entity_pair,([s_positon_start,s_positon_end], [t_positon_start,t_positon_end]))
print(len(entity_pair_dict))
# print(entity_pair_dict)

# Basic rule for generate data
# First, if (there are numbers in sentences):
#     Next if (a, p) in sentences:
#         Next sentences exist {'rel': (entity_1, entity_2)}

# print(set(st_relation_list))

('fold', '40') ([26, 28], [29, 33])
('cross-validation', 'fold') ([29, 33], [34, 50])
('cross-validation', 'fold') ([29, 33], [34, 50])
('64', 'hyperparameter search') ([111, 132], [161, 163])
('batch size', '64') ([161, 163], [147, 157])
('\\lbrace 1e-5, 2e-5, 3e-5\\rbrace', 'hyperparameter search') ([111, 132], [189, 220])
('learning rates', '\\lbrace 1e-5, 2e-5, 3e-5\\rbrace') ([189, 220], [168, 182])
('\\lbrace 8,16,64,128\\rbrace', 'hyperparameter search') ([111, 132], [246, 272])
('epochs number', '\\lbrace 8,16,64,128\\rbrace') ([246, 272], [226, 239])
('0.01', 'hyperparameter search') ([111, 132], [350, 354])
('weight decay coefficient', '0.01') ([350, 354], [322, 346])
('6%', 'hyperparameter search') ([111, 132], [306, 308])
('steps', '6%') ([306, 308], [312, 317])
('linear warmup', 'steps') ([312, 317], [278, 291])
('1e-5', 'each dataset') ([462, 474], [450, 454])
('learning rates', '1e-5') ([450, 454], [168, 182])
('8', '1,000 elements datasets') ([493, 516], [480, 481])
('w

In [7]:
a_entity = set(entity_type['a'])
p_entity = set(entity_type['p'])
c_entity = set(entity_type['c'])
print('number of a type: ', len(a_entity))
print('number of p type: ', len(p_entity))
print('number of c type: ', len(c_entity))

number of a type:  28
number of p type:  44
number of c type:  38


In [11]:
ap_rel = set(rel_to_entity['ap'])
pv_rel = set(rel_to_entity['pv'])
vc_rel = set(rel_to_entity['vc'])
print('number of ap type: ', len(ap_rel))
print('number of pv type: ', len(pv_rel))
print('number of vc type: ', len(vc_rel))

number of ap type:  54
number of pv type:  77
number of vc type:  64


In [26]:
len(ap_rel)

54

In [12]:
ap_rel

{('Adam', 'epochs'),
 ('Adam', 'learning rate'),
 ('Adam', 'patience value'),
 ('Adam', 'weight decay'),
 ('AdamW', 'batch size'),
 ('AdamW', 'learning rates'),
 ('BiLSTM', 'dimensional hidden layers'),
 ('BiLSTM', 'hidden state'),
 ('BiLSTM', 'layer'),
 ('COCI (OpenCitations Index of Crossref open DOI-to-DOI references)',
  'Version'),
 ('CRPSE', 'K'),
 ('EMA', 'm'),
 ('FFNNs', 'dimensional hidden layers'),
 ('FFNNs', 'dropout'),
 ('FFNNs', 'hidden layers'),
 ('GNN', 'layers'),
 ('L', '\\alpha'),
 ('L', '\\alpha _1'),
 ('L', '\\alpha _2'),
 ('LSTMs', 'variational dropout'),
 ('Prefix beam search decoding', 'beam-width'),
 ('ProxyAnchor Loss', '\\alpha'),
 ('ProxyAnchor Loss', '\\delta'),
 ('ProxyNCA Loss', '\\delta'),
 ('SoftTriple loss,', '\\delta'),
 ('SoftTriple loss,', '\\gamma'),
 ('SoftTriple loss,', '\\lambda'),
 ('SoftTriple loss,', '\\tau'),
 ('SupCon Loss', '\\tau'),
 ('Triplet Loss', 'm'),
 ('W&B Sweeps', 'counts'),
 ('\\sigma', 'T'),
 ('beam pruning', '\\lambda _{\\text{C}

In [22]:
## full triples
apv_set = 0
paragraph_set = 0
gt_data = []
for paragraph in joint_dataset.keys():
    params2nums = {'a':0, 'p':0, 'v':0, 'c':0}
    for rel in joint_dataset[paragraph]['relation'].keys():
        source = joint_dataset[paragraph]['relation'][rel]['source'][0]
        target = joint_dataset[paragraph]['relation'][rel]['target'][0]
        params2nums[source] += 1
        params2nums[target] += 1
    if params2nums['a']!=0 and params2nums['p']!=0 and params2nums['v']!=0:
        paragraph_set += 1
        params2nums.pop('c')
        min_ = min(params2nums.values())
        #print(params2nums, min_)
        apv_set += min_
        # save data with 2 sets (a p v)
        gt_data.append(joint_dataset[paragraph])
#         print(params2nums)

print('relation with at least two sets(a p v):', apv_set)
print('paragraph with at least two sets(a p v):', paragraph_set)
# print('left: ', len(gt_data))
# print(gt_data)

relation with at least two sets(a p v): 46
paragraph with at least two sets(a p v): 13


In [25]:
# save to JSON file
with open("ap_rel.json", "w") as json_file:
    json.dump(list(ap_rel), json_file)